LDA= Statistical model for discovering abstract collection of topics within a document
* Docs with similar topics use similar group of words
* Latent topics are doscovered by groups of words that occur together in docs

* LDA suggests that documents are probabilty distribution of related topics
* Topics are probabilty distribution of words

Doc1 = 0.4 Topic1+ ....

Doc2 = 0.3 Topic 2+


In [2]:
import gensim
import os
import collections
import smart_open
import random


In [3]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

In [4]:
gensim.__path__[0]

'/Users/deepak/anaconda/lib/python3.6/site-packages/gensim'

In [5]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [6]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))


In [7]:
train_corpus[:2]


[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

In [8]:
print(test_corpus[:2])


[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist'], ['cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to'

In [9]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)

In [10]:
model.build_vocab(train_corpus)

In [11]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)


CPU times: user 4.93 s, sys: 285 ms, total: 5.22 s
Wall time: 2.31 s


2347351

In [12]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])


array([ 0.0526687 , -0.13740814, -0.10726429, -0.06315795,  0.00396701,
        0.01943713,  0.0287191 , -0.01192777,  0.00658986,  0.0821916 ,
        0.01717857,  0.06607112, -0.01336175, -0.08826929,  0.15613106,
        0.00985721,  0.03872054,  0.10129673,  0.02046631, -0.0843548 ,
       -0.05616848,  0.007538  ,  0.0205717 ,  0.06196138, -0.06001822,
       -0.07432699, -0.07344505, -0.0696364 , -0.03536064, -0.01116091,
        0.0113153 , -0.10564786,  0.01341553,  0.06552637, -0.05595791,
        0.00607053, -0.06808197, -0.04142086,  0.00432042,  0.06605968,
        0.03829645, -0.17863749,  0.03016104,  0.05081179, -0.07734151,
        0.02107528,  0.08213865, -0.02619808, -0.0730366 , -0.13211657], dtype=float32)